In [1]:
from transformers import BartTokenizer,BartConfig
tokeniser = BartTokenizer.from_pretrained('facebook/bart-base')
tokeniser.add_tokens(['[SEP]'])
tokeniser.get_added_vocab()['[SEP]']

50265

In [ ]:
tokeniser.sep_token_id

In [2]:
from src.contextual_bart import ContextualisedBartModel,BartForContextualRecovery

In [3]:
bart_config = BartConfig.from_pretrained('facebook/bart-base')
bart_config.context_delimiter_id = tokeniser.get_added_vocab()['[SEP]']

In [4]:
con_bart = BartForContextualRecovery.from_pretrained('facebook/bart-base',
                                                   config = bart_config,
                                                   ignore_mismatched_sizes=True
                                                   )

In [5]:
con_bart.resize_token_embeddings(len(tokeniser))

Embedding(50266, 768)

In [6]:
oop_tok = tokeniser(['He was here [SEP] boy friend.',
                     "We can find him because [SEP] he was"],
                    return_tensors="pt",padding=True)
oop = con_bart(**oop_tok)

EncoderOutputs(last_hidden_state=tensor([[[ 0.0504,  0.0618,  0.3333,  ...,  0.1143, -0.0497,  0.1502],
         [-0.1628, -0.2547,  0.1759,  ...,  0.0351,  0.3040,  0.1871],
         [-0.0157,  0.1318,  0.1969,  ...,  0.1808, -0.1956,  0.2334],
         [-0.1582, -0.0009,  0.2791,  ...,  0.1717, -0.1716,  0.4238],
         [ 0.0728,  0.1454,  0.1331,  ...,  0.1525, -0.0421,  0.0894]],

        [[-0.0839,  0.0606,  0.1464,  ...,  0.2486,  0.1477,  0.1590],
         [ 0.0529,  0.1785, -0.1232,  ...,  0.1522, -0.0398,  0.1828],
         [-0.1257, -0.0641,  0.5198,  ...,  0.2859, -0.2460,  0.5156],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<CatBackward0>), hidden_states=None, attentions=None, attention_mask=tensor([[1., 1., 1., 1., 0.],
        [1., 1., 1., 0., 0.]]))
torch.Size([2, 5, 768])  huye
torch.Size([2, 5])  Meeu


torch.Size([2, 5, 768])

In [ ]:
from src.dataset_processor import ContextGenerationDataset, load_all_data

In [ ]:
train_data = load_all_data('processed_data/',)
dev_data = load_all_data('processed_data/','dev')

In [ ]:
for dat in train_data:
    if '[SEP]' not in dat.input:
        print(dat)

In [ ]:
train_data[0]

In [ ]:
train_dataset = ContextGenerationDataset(tokeniser,)
train_dataset.set_record(train_data)

dev_dataset = ContextGenerationDataset(tokeniser,)
dev_dataset.set_record(dev_data)

In [ ]:
context_delimiter_id = tokeniser.get_added_vocab()['[SEP]']
def _strip_context(input_ids, ):
        """

        :param input_ids:
        :param embeddings:
        :param attention_mask:
        :return:
        """

        # identify the locations of the context_delimiter in each of the input sequence
        if type(input_ids) is list:
            input_ids = torch.LongTensor(
                input_ids,
            )
        delimiter_points = input_ids == context_delimiter_id
        delimiter_points_idxs = delimiter_points.nonzero(as_tuple=True)[-1]
        
        if len(delimiter_points_idxs)!=input_ids.shape[0]:
            print(delimiter_points_idxs)
            return False
        return True

In [ ]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler
from src.utils import SmartCollator

In [ ]:
smart_collator=SmartCollator(pad_token_id=train_dataset.tokenizer.pad_token_id, 
                                    max_len=700)

In [ ]:
data_loader = DataLoader(train_dataset,batch_size=16,
                         sampler=SequentialSampler(train_dataset),
                         collate_fn=smart_collator
                         )

In [ ]:
context_delimiter_id

In [ ]:
tokeniser.decode(dat['input_ids'][15])

In [ ]:
dat['input_ids'].shape

In [ ]:
[idx for idx,d in enumerate(dat['input_ids']) if 50265 not in d]

In [ ]:
for idx,dat in enumerate(data_loader):
    if not _strip_context(dat['input_ids']):
        for v,d in enumerate(dat):
            _strip_context(d)
        print(idx)
    